<a href="https://colab.research.google.com/github/Bhairavi-shah/AbstractiveUserReviewConsolidation/blob/master/Module4_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Initialise drive configurations
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/My Drive/PROJECT | S7-S8/Colab Notebooks/')

#import custom attention layer
from attention import AttentionLayer

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#Import modules
import numpy as np
import pandas as pd 
import re
import pickle
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from google.colab import files
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
%%time
#Read data
data = pd.read_csv(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/training_seq2seq.csv")
print(data.columns)
print(data)

Index(['Unnamed: 0', 'Id', 'ProductId', 'UserId', 'ProfileName',
       'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time',
       'Summary', 'Text'],
      dtype='object')
        Unnamed: 0  ...                                                                                                                                                                                                     Text
0           562970  ...                                                                  I love these cookies!  Not only are they healthy but they taste great and are so soft!  I will definitely add these to my grocery list!
1           562971  ...  Quaker Soft Baked Oatmeal Cookies with raisins are a delicious treat, great for anytime of day.  For example:<br /><br />--at breakfast, I had one with a large banana and a cup of coffee, and felt...
2           562972  ...  I am usually not a huge fan of oatmeal cookies, but these literally melt in your mouth. They are so soft and tas

In [0]:
#dropping duplicates and na values
data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,inplace=True)

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [6]:
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) 

# Preprocessing function
def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#call text_cleaner function to preprocess both review and summary for training
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0)) 

cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [0]:
#drop empty rows
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [9]:
#to fix max length of the sequence
cnt=0
for i in data['cleaned_summary']:
    if(len(i.split())<=15):
        cnt=cnt+1
print(cnt/len(data['cleaned_summary']))

max_text_len=50
max_summary_len=15

0.9978761437850657


In [0]:
#selecting reviews with length equal to or less than max_len
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'text':short_text,'summary':short_summary})

#adding start and end tokens to the data
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [0]:
#split train and test data
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True) 

In [0]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [13]:
#find proportion of rare words in the data
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 63.80149341387701
Total Coverage of rare words: 2.3025567271771283


In [0]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr)
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

In [0]:
#prepare a tokenizer for summary on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [16]:
#find proportion of rare words in the data
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 74.65366129488267
Total Coverage of rare words: 4.417379107337768


In [0]:
#prepare a tokenizer for summary on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1

In [0]:
#delete rows with only start and end tokens
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [0]:
#Save the tokenizers to be used for prediction
#with open('text_tokenizer.pickle', 'wb') as handle:
#    pickle.dump(x_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
#files.download('text_tokenizer.pickle')
#with open('summary_tokenizer.pickle', 'wb') as handle:
#    pickle.dump(y_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
#files.download('summary_tokenizer.pickle')

In [20]:
#from keras import backend as K
from tensorflow.keras import backend as K
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary() 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 100)      863000      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 50, 300), (N 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [21]:
%%time
#compile and train the model
filename = 'model.h5'
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=100,batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/100
272/272 [==============================] - 160s 589ms/step - loss: 1.7013 - val_loss: 1.4671
Epoch 2/100
272/272 [==============================] - 159s 583ms/step - loss: 1.4679 - val_loss: 1.3856
Epoch 3/100
272/272 [==============================] - 158s 580ms/step - loss: 1.3809 - val_loss: 1.3193
Epoch 4/100
272/272 [==============================] - 158s 581ms/step - loss: 1.3109 - val_loss: 1.2711
Epoch 5/100
272/272 [==============================] - 157s 577ms/step - loss: 1.2572 - val_loss: 1.2333
Epoch 6/100
272/272 [==============================] - 158s 580ms/step - loss: 1.2144 - val_loss: 1.2062
Epoch 7/100
272/272 [==============================] - 157s 577ms/step - loss: 1.1774 - val_loss: 1.1944
Epoch 8/100
272/272 [==============================] - 157s 577ms/step - loss: 1.1470 - val_loss: 1.1751
Epoch 9/100
272/272 [==============================] - 156s 574ms/step - loss: 1.1189 - val_loss: 1.1623
Epoch 10/100
272/272 [==============================] -

In [0]:
#save the model
model.save("seq2seq.h5")
#files.download('seq2seq.h5')

In [0]:
#dictionary to convert the index to word for target and source vocabulary
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

**Inference Phase**

In [0]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [27]:
#Testing
for i in range(0,100):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

Review: tried almost happy baby flavors honestly say one daughter favorite flavor brand wish package slightly larger able finish moments sometimes wants second 
Original summary: our favorite happy baby flavor 
Predicted summary:  our favorite happy baby loves it


Review: illy issimo coffee drink cappuccino treat hard day away computer filled tall glass crushed ice poured smooth creamy cappuccino sipped away actually took break feet settling back chair closing eyes rich coffee cocoa goodness ate ice 
Original summary: delicious treat only calories 
Predicted summary:  coffee for cold cappuccino coffee drink


Review: love nuts tired snack bars oats featured ingredients much sugary sweet packed big crunchy chunks honey roasted peanuts almonds cashews enough rice granola together perfect balance sweet salty love first bite bar calories fiber protein fat sugar cholesterol carbs 
Original summary: yes these are really good 
Predicted summary:  snack bar


Review: goji berries great tastin

In [28]:
%%time
import nltk
nltk.download('punkt')
def sum_extract(all_summaries):
  article_text = all_summaries
  formatted_article_text = all_summaries
  sentence_list = nltk.sent_tokenize(article_text)
  stopwords = nltk.corpus.stopwords.words('english')

  word_frequencies = {}
  for word in nltk.word_tokenize(formatted_article_text):
      if word not in stopwords:
          if word not in word_frequencies.keys():
              word_frequencies[word] = 1
          else:
              word_frequencies[word] += 1

  maximum_frequncy = max(word_frequencies.values())

  for word in word_frequencies.keys():
      word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

  sentence_scores = {}
  for sent in sentence_list:
      for word in nltk.word_tokenize(sent.lower()):
          if word in word_frequencies.keys():
              if len(sent.split(' ')) < 30:
                  if sent not in sentence_scores.keys():
                      sentence_scores[sent] = word_frequencies[word]
                  else:
                      sentence_scores[sent] += word_frequencies[word]

  import heapq
  summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)

  summary = ' '.join(summary_sentences)
  print(summary)
  return(summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CPU times: user 289 ms, sys: 47.2 ms, total: 336 ms
Wall time: 457 ms


In [29]:
%%time
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
pd.set_option('display.max_colwidth',1000)

import nltk
import pickle
import re, string, random
from nltk.tag import pos_tag
from google.colab import files
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk import FreqDist, classify, NaiveBayesClassifier

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('averaged_perceptron_tagger')

from bs4 import BeautifulSoup
import urllib3
import json
http=urllib3.PoolManager()
Abbr_dict={}

#Function to get the Slangs from https://www.noslang.com/dictionary/
def getAbbr(alpha):
    global Abbr_dict
    r=http.request('GET','https://www.noslang.com/dictionary/'+alpha)
    soup=BeautifulSoup(r.data,'html.parser')
    
    for i in soup.findAll('div',{'class':'dictionary-word'}): 

        abbr=i.find('abbr')['title']
        Abbr_dict[i.find('span').text[:-2]]=abbr

#Generating a-z
linkDict=[]
for x in range(97,123):
    linkDict.append(chr(x))

#Creating Links for https://www.noslang.com/dictionary/a...https://www.noslang.com/dictionary/b....etc
for i in linkDict:
    getAbbr(i)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
CPU times: user 2.75 s, sys: 209 ms, total: 2.96 s
Wall time: 12.5 s


In [30]:
%%time
def remove_noise(tweet_tokens, stop_words = ()):
  cleaned_tokens = []
  for token, tag in pos_tag(tweet_tokens):
    token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
    token = re.sub("(@[A-Za-z0-9_]+)","", token)
    if token in Abbr_dict.keys():
      token = Abbr_dict[token]
    if tag.startswith("NN"):
      pos = 'n'
    elif tag.startswith('VB'):
      pos = 'v'
    else:
      pos = 'a'
    lemmatizer = WordNetLemmatizer()
    token = lemmatizer.lemmatize(token, pos)
    if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
      cleaned_tokens.append(token.lower())
  return cleaned_tokens

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [31]:
%%time
import pandas as pd
pickle_in = open(r"/content/gdrive/My Drive/PROJECT | S7-S8/Pickle Files/nb_sentiment_analysis_final.pkl","rb")
classifier = pickle.load(pickle_in)
def pred_sentiment(sum_text):
  sum_text = remove_noise(word_tokenize(sum_text))
  sentiment = classifier.classify(dict([tok, True] for tok in sum_text))
  return(sentiment)

CPU times: user 330 ms, sys: 2.53 ms, total: 333 ms
Wall time: 2.18 s


In [34]:
%%time
from gensim.summarization.summarizer import summarize 
from gensim.summarization import keywords
data = pd.read_csv(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/testing_seq2seq.csv")
meta_data = pd.read_json("/content/gdrive/My Drive/PROJECT | S7-S8/Data/product_details (1).json")

#dropping duplicates and na values
data.drop_duplicates(subset=['Text'],inplace=True)
data.dropna(axis=0,subset=['Text'], inplace=True)

#call text_cleaner function to preprocess both review and summary for training
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0)) 

data['cleaned_text']=cleaned_text

data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

res_df = pd.DataFrame(columns=['pid', 'title', 'summaries', 'avg_rating', 'sentiment'])

j = 0

pid_test = ['B000CQE3NM', 'B001L4JH5I', 'B003EM7J9Q', 'B000CQID2Y', 'B005K4Q1T0', 'B000DZFMEQ', 'B001GL6GBE', 'B003CK7O36', 'B003QNJYXM', 'B000ZSZ5S4', 'B001EO5Y8Y', 'B000CQC08C', 'B0039ZOZ86', 'B005K4Q37A', 'B000REI2X6', 'B000PDY3P0', 'B005ZBZLSU', 'B002GJ9JY6', 'B006N3HYYS', 'B000CQIDHY', 'B001E5E3JY', 'B000F6SNPS', 'B005ZBZLT4', 'B000CQIDHO', 'B0008IT4OM', 'B003CK2BQG', 'B000ESLJ6C', 'B000HDK0DC', 'B001EYUE5M', 'B005DFL4PM', 'B001CWX7EG', 'B000DZDJ0K', 'B000CQBZOW', 'B004SRH2B6', 'B005GX00BK', 'B007TJGZ5E', 'B008ZRKZSM', 'B000CQG8KS', 'B000CQC05K', 'B0058AMYTC', 'B005HG9ESG', 'B000GAT6NG', 'B000CQIDHE', 'B000WB1YSE', 'B0012BUR8Q', 'B0098WV8F2', 'B002AQP5MK', 'B005GTWCTM', 'B001LG940E', 'B0033HPPIO']

for pid in pid_test:
  row = [pid, ]
  df = data.loc[data['ProductId'] == pid]
  df2 = meta_data.loc[meta_data['asin'] == pid]
  row += [df2['title'].values[0],]
  x_tr = np.array(df['cleaned_text'])

  #prepare a tokenizer for reviews on training data
  x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
  x_tokenizer.fit_on_texts(list(x_tr))

  #convert text sequences into integer sequences
  x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr)

  #padding zero upto maximum length
  x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')

  #size of vocabulary ( +1 for padding token)
  x_voc   =  x_tokenizer.num_words + 1

  sums = []
  sents = []
  for i in range(len(x_tr)):
    try:
      s = decode_sequence(x_tr[i].reshape(1,max_text_len))
    except:
      print("Key Error occurred")
      continue
    if pred_sentiment(seq2text(x_tr[i])) == 'Positive':
      sents += [1, ]
    else :
      sents += [0, ]
#    print("Review:",seq2text(x_tr[i]))
#    print("Predicted summary:",s)
    sums += [s,]
#    print("\n")
  row += [". ".join(sums),]
  row += [df['Score'].sum()/len(df),]
  if sents.count(1) > sents.count(0):
    row += [1,]
  else:
    row += [0,]
  res_df = res_df.append({'pid':row[0], 'title': row[1], 'summaries':summarize((row[2]), ratio = 0.01).replace('\n', ' '), 'avg_rating':row[3], 'sentiment':row[4]},ignore_index=True)
  print(res_df.loc[j])
  j += 1
#  print(res_df)

print(res_df)

pid                                                                                                                                                                                                        B000CQE3NM
title         Stash Tea Moroccan Mint Green Tea 20 Count Box of Tea Bags (Pack of 6), Tea Bags Individually Wrapped in Foil (packaging may vary), Medium Caffeine Tea, Green Tea Blended with Mint, Drink Hot or Iced
summaries                                                                                                                                                                                                            
avg_rating                                                                                                                                                                                                        NaN
sentiment                                                                                                                                       

In [35]:
res_df.loc[0]

pid                                                                                                                                                                                                        B000CQE3NM
title         Stash Tea Moroccan Mint Green Tea 20 Count Box of Tea Bags (Pack of 6), Tea Bags Individually Wrapped in Foil (packaging may vary), Medium Caffeine Tea, Green Tea Blended with Mint, Drink Hot or Iced
summaries                                                                                                                                                                                                            
avg_rating                                                                                                                                                                                                        NaN
sentiment                                                                                                                                       

In [36]:
new_js = res_df.to_json ('result.json', orient='records')
print(new_js)

None
